In [ ]:
!pip install l5kit
!pip install -U PyYAML

In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models.git
!pip install --target=/kaggle/working pymap3d==2.1.0 -q
!pip install --target=/kaggle/working strictyaml -q
!pip install --target=/kaggle/working protobuf==3.12.2 -q
!pip install --target=/kaggle/working transforms3d -q
!pip install --target=/kaggle/working zarr -q
!pip install --target=/kaggle/working ptable -q
!pip install --no-dependencies --target=/kaggle/working l5kit==1.1.0 --upgrade -q
!pip install pytorch-pfn-extras==0.3.1

  Cloning https://github.com/rwightman/pytorch-image-models.git to /tmp/pip-req-build-4umz9dye
  Running command git clone -q https://github.com/rwightman/pytorch-image-models.git /tmp/pip-req-build-4umz9dye
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

In [ ]:
!pip uninstall opencv-python-headless==4.5.5.62

!pip install opencv-python-headless==4.1.2.30

Found existing installation: opencv-python-headless 4.1.2.30
Uninstalling opencv-python-headless-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.1.2.30.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/cv2/.libs/libQtCore-bbdab771.so.4.8.7
    /usr/local/lib/python3.7/dist-packages/cv2/.libs/libQtGui-903938cd.so.4.8.7
    /usr/local/lib/python3.7/dist-packages/cv2/.libs/libQtTest-1183da5d.so.4.8.7
    /usr/local/lib/python3.7/dist-packages/cv2/config-3.py
    /usr/local/lib/python3.7/dist-packages/cv2/config.py
    /usr/local/lib/python3.7/dist-packages/cv2/cv2.abi3.so
    /usr/local/lib/python3.7/dist-packages/cv2/gapi/__init__.py
    /usr/local/lib/python3.7/dist-packages/cv2/load_config_py2.py
    /usr/local/lib/python3.7/dist-packages/cv2/load_config_py3.py
    /usr/local/lib/python3.7/dist-packages/cv2/mat_wrapper/__init__.py
    /usr/lo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install timm

In [ ]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/kaggle.json
!echo '{"username":"simarkareer","key":"ff816530aeb5eda4d7ce160a471cbe14"}' >> /root/.kaggle/kaggle.json
!cat /root/.kaggle/kaggle.json
!mkdir lyftdataset
!cd lyftdataset
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle --version

{"username":"simarkareer","key":"ff816530aeb5eda4d7ce160a471cbe14"}
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=db640b380b30e1705b297e5a3550e817944770db29a098cfcaa82ef0097b9eb3
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Kaggle API 1.5.12


In [ ]:
%cd lyftdataset
!unzip -q /content/drive/MyDrive/Deep\ Learning\ Project/lyft-motion-prediction-autonomous-vehicles.zip

[Errno 2] No such file or directory: 'lyftdataset'
/content/lyftdataset


In [ ]:
!wget https://raw.githubusercontent.com/lyft/l5kit/master/examples/agent_motion_prediction/agent_motion_config.yaml


--2022-04-28 19:01:45--  https://raw.githubusercontent.com/lyft/l5kit/master/examples/agent_motion_prediction/agent_motion_config.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2019 (2.0K) [text/plain]
Saving to: ‘agent_motion_config.yaml’

agent_motion_config 100%[===================>]   1.97K  --.-KB/s    in 0s      

2022-04-28 19:01:45 (29.8 MB/s) - ‘agent_motion_config.yaml’ saved [2019/2019]



In [ ]:
from timm.models import vision_transformer
import torch
import l5kit, os
import torch.nn as nn
import numpy as np
import warnings;warnings.filterwarnings("ignore")
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from tqdm import tqdm
from l5kit.geometry import transform_points
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable
import torch.optim as optim
import torch.nn.functional as F
from l5kit.evaluation import write_pred_csv
from l5kit.geometry import transform_points
# cfg = load_config_data("../input/lyft-config-files/agent_motion_config.yaml")
cfg = load_config_data("/content/lyftdataset/agent_motion_config.yaml")
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
model = vision_transformer.vit_small_patch32_224(pretrained=True)

In [ ]:
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from torch.utils.data import DataLoader

dm = LocalDataManager()
test_config = cfg["test_data_loader"]
# test_config = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
test_chunked = ChunkedDataset(dm.require(test_config["key"])).open()
test_mask = np.load(f"../input/lyft-motion-prediction-autonomous-vehicles/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_chunked, rasterizer, agents_mask=test_mask)
test_loader = torch.utils.data.DataLoader(test_dataset,
                              shuffle=test_config["shuffle"],
                              batch_size=test_config["batch_size"],
                              num_workers=test_config["num_workers"])



FileNotFoundError: ignored

In [ ]:
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
dm = LocalDataManager()
train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                              shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"],
                              num_workers=train_cfg["num_workers"])

val_cfg = cfg["val_data_loader"]
val_zarr = ChunkedDataset(dm.require(val_cfg["key"])).open()
val_dataset = AgentDataset(cfg, val_zarr, rasterizer)
val_dataset = torch.utils.data.Subset(val_dataset, range(0, 4000))
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                              shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"],
                              num_workers=train_cfg["num_workers"])

FileNotFoundError: ignored

In [ ]:
# ===== INIT DATASET
train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
print(train_dataset)

In [ ]:
class LyftVIT(nn.Module):
    
    def __init__(self, vit: nn.Module):
        super(LyftVIT, self).__init__()
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        self.vit = vit
        num_targets = 2 * cfg["model_params"]["future_num_frames"]
        self.future_len = cfg["model_params"]["future_num_frames"]
        self.vit.patch_embed.backbone.conv1[0] = nn.Conv2d(
            num_in_channels,
            32,
            kernel_size=self.vit.patch_embed.backbone.conv1[0].kernel_size,
            stride=self.vit.patch_embed.backbone.conv1[0].stride,
            padding=self.vit.patch_embed.backbone.conv1[0].padding,
            bias=False,
        )
        
        
        self.num_preds = num_targets * 3
        self.num_modes = 3
        
        self.logit = nn.Linear(1000, out_features=self.num_preds + self.num_modes)
        
    def forward(self, x):
        x = self.vit(x)
        x = torch.flatten(x, 1)
        x = self.logit(x)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences

In [ ]:
model = LyftVIT(model)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction="none")

In [ ]:
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch["image"].to(device), batch["target_positions"].to(device)
    avails = batch["target_availabilities"].unsqueeze(-1).to(device)
    y_pred, conf = model(x)
    loss = pytorch_neg_multi_log_likelihood_batch(y, y_pred, conf, avails[:, :, 0])
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(image, target_positions, **kwargs):
    x, y = image.to(device), target_positions.to(device)
    y_pred = model(x)
    return y_pred, y

In [ ]:
valid_evaluator = E.Evaluator(
    val_dataloader,
    model,
    progress_bar=False,
    eval_func=validation_step,
)

log_trigger = (1000, "iteration")
log_report = E.LogReport(trigger=log_trigger)
extensions = [
    log_report,  # Save `log` to file
    valid_evaluator,
    # E.FailOnNonNumber()  # Stop training when nan is detected.
    E.ProgressBarNotebook(update_interval=100),  # Show progress bar during training
    E.PrintReportNotebook(),  # Show "log" on jupyter notebook  
]

In [ ]:
models = {"main": model}
optimizers = {"main": optimizer}
manager = IgniteExtensionsManager(
    trainer,
    models,
    optimizers,
    7,
    extensions=extensions,
    out_dir="../working",
)
manager.extend(E.snapshot_object(model, "predictor.pt"),
               trigger=(50, "iteration")) 

In [ ]:
manager.iteration = 0
manager._iters_per_epoch = len(train_dataloader)
trainer.run(train_dataloader, max_epochs=7)

In [ ]:
model.vit.patch_embed.backbone.conv1[0] = nn.Conv2d(
            5,
            32,
            kernel_size=model.vit.patch_embed.backbone.conv1[0].kernel_size,
            stride=model.vit.patch_embed.backbone.conv1[0].stride,
            padding=model.vit.patch_embed.backbone.conv1[0].padding,
            bias=False,
        )
model.load_state_dict(torch.load('../input/lyft-vision-transformer-training/predictor.pt'))
model.vit.patch_embed.backbone.conv1[0] = nn.Conv2d(
            25,
            32,
            kernel_size=model.vit.patch_embed.backbone.conv1[0].kernel_size,
            stride=model.vit.patch_embed.backbone.conv1[0].stride,
            padding=model.vit.patch_embed.backbone.conv1[0].padding,
            bias=False,
        ).to(device)

In [ ]:
model.eval()

future_coords_offsets_pd = []
timestamps = []
agent_ids = []
confs = []

with torch.no_grad():
    dataiter = tqdm(test_loader)
    
    for data in dataiter:

        inputs = data["image"].to(device)
        target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
        targets = data["target_positions"].to(device)
        
        outputs, conf = model(inputs)
        preds = outputs.cpu().numpy()
        conf = conf.cpu().numpy()
        world_from_agents = data["world_from_agent"].numpy()
        centroids = data["centroid"].numpy()
        coords_offset = []
        
        # convert into world coordinates and compute offsets
        for idx in range(len(preds)):
            for mode in range(3):
                preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]
    
        future_coords_offsets_pd.append(preds.copy())
        confs.append(conf.copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy()) 

In [ ]:
write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
              confs=np.concatenate(confs))